In [1]:
!pip install -U sentence-transformers
!pip install faiss-cpu --upgrade
!pip install faiss-gpu --upgrade
!pip install text_generation
!pip install gradio
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ad4d243ba2eb26034c5d45f6b3e71e2e1c35dfedcca17c4de00cbc0c6f69d7dd
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, StoppingCriteria, TextIteratorStreamer, TextStreamer
import torch
import gradio as gr

model_name = "kfkas/Llama-2-ko-7b-Chat"

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map={'':0})

model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)

retriever = SentenceTransformer('jhgan/ko-sroberta-multitask', device='cpu')

df = pd.read_csv("/content/gdrive/MyDrive/melon_popular_songs.csv")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
class MyStoppingCriteria(StoppingCriteria):
  def __init__(self, target_sequence, prompt):
      self.target_sequence = target_sequence
      self.prompt=prompt

  def __call__(self, input_ids, scores, **kwargs):
      # Get the generated text as a string
      generated_text = tokenizer.decode(input_ids[0])
      generated_text = generated_text.replace(self.prompt,'')
      # Check if the target sequence appears in the generated text

      for target in self.target_sequence:
        if target in generated_text:
            return True  # Stop generation

      return False  # Continue generation

  def __len__(self):
      return 1

  def __iter__(self):
      yield self

In [6]:
def get_embeddings(texts):
    return retriever.encode(texts, convert_to_tensor=True)

tag_embeddings = get_embeddings(df['tag'])

index = faiss.IndexFlatIP(tag_embeddings.size(-1))
index.add(tag_embeddings.numpy())

In [7]:
def get_songs(query):
  query_embedding = get_embeddings([query])

  D, I = index.search(query_embedding.numpy(), k=5)

  if D[0][0] < 35.0:
    return None

  top_5_songs = df.iloc[I[0]]

  top_5_results = []

  for i, row in top_5_songs.iterrows():
      name = row['name']
      singer = row['singer']
      top_5_results.append(f"{name} - {singer}")

  return top_5_results

In [8]:
def format_chat_prompt(message, chat_history, songs=None):
    if songs:
        prompt = """
지시문:
- 사용자에게 공감을 해준다.
- 사용자에게 노래를 추천한다.

사용자: 기분이 우울해..
노래: 같이 걷자 - ['선우정아']
답변: 저런, 기분이 우울하시군요. 당신에게 도움이 될 만한 노래를 찾아보았어요. 선우정아의 '같이 걷자'를 들어보시는 것은 어떨까요? 당신에게 힘이 되면 좋겠어요.

사용자: 드라이브할 때 듣기 좋은 노래가 있을까?
노래: 여행 - ['볼빨간사춘기']
답변: 드라이브를 가시는군요! 드라이브에는 설렘이 가득한 노래가 좋을 것 같아요. 볼빨간사춘기의 '여행'을 추천드려요.

  """
        message = f"{message}\n노래: {songs[0]}"

    else:
        prompt = "지시문:\n사용자의 요청에 맞는 답변을 생성하세요. 친절하고 부드러운 말투를 사용하세요.\n"
    # for turn in chat_history:
    #     user_message, bot_message = turn
    #     prompt = f"{prompt}\n사용자: {user_message}\n답변: {bot_message}"
    prompt = f"{prompt}\n사용자: {message}\n답변:"
    return prompt

In [9]:
def respond(message, chat_history, temperature=0.8):

    songs = get_songs(message)

    prompt = format_chat_prompt(message, chat_history, songs=songs)
    chat_history = chat_history + [[message, ""]]
    encoded_input = tokenizer(prompt, return_tensors='pt')
    input_ids=encoded_input['input_ids'].cuda()
    streamer = TextIteratorStreamer(tokenizer, timeout=20., skip_prompt=True, skip_special_tokens=True)

    generated_text = model.generate(input_ids,
                    streamer=streamer,
                    pad_token_id=tokenizer.eos_token_id,
                    do_sample=True,
                    max_new_tokens=64,
                    repetition_penalty=1.1,
                    top_k=20,
                    top_p=0.92,
                    temperature=temperature,
                    stopping_criteria=MyStoppingCriteria(["사용자:", "<|endoftext|>", "\n", "\u200b"], prompt)
                    )

    model_output = ""
    for new_text in streamer:
        model_output += new_text

    if songs:
      model_output += "\n그밖에도"
      for song in songs[1:]:
        model_output += "\n\n" + song

      model_output += "\n\n등의 음악이 있습니다."

    # stop_sequences to not generate the user answer
    acc_text = ""
    # Streaming the tokens
    for idx, response in enumerate(model_output):
        text_token = response

        if idx == 0 and text_token==" ":
            text_token = text_token[1:]

        acc_text += text_token
        last_turn = list(chat_history.pop(-1))
        last_turn[-1] += acc_text
        chat_history = chat_history + [last_turn]
        yield "", chat_history
        acc_text = ""

In [10]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=480)  # just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    with gr.Accordion(label="Advanced options", open=False):
        system = gr.Textbox(
            label="System message",
            lines=2,
            value="A conversation between a user and an LLM-based AI assistant. The assistant gives helpful and honest answers.",
        )
        temperature = gr.Slider(
            label="temperature", minimum=0.1, maximum=1, value=0.7, step=0.1
        )
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(
        respond, inputs=[msg, chatbot], outputs=[msg, chatbot]
    )  # Press enter to submit
gr.close_all()
demo.queue().launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e6c8b05ee0bd2a047e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
